In [1]:
import numpy as np
import os
np.random.seed(123)
from six.moves import cPickle

from keras import backend as K
from keras.models import Model
from keras.layers import Input, Dense, Flatten
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.optimizers import Adam

from prednet import PredNet
from data_utils import SequenceGenerator

Using TensorFlow backend.


In [2]:
WEIGHTS_DIR = './weights/'
DATA_DIR = '../data/'

In [3]:
save_model = True  # if weights will be saved
weights_file = os.path.join(WEIGHTS_DIR, 'prednet_weather_weights.hdf5')  # where weights will be saved
json_file = os.path.join(WEIGHTS_DIR, 'prednet_weather_model.json')

In [4]:
# Data files
train_file = os.path.join(DATA_DIR, 'x_train.hkl')
train_sources = os.path.join(DATA_DIR,  'sources_train.hkl')
val_file = os.path.join(DATA_DIR, 'x_val.hkl')
val_sources = os.path.join(DATA_DIR, 'sources_val.hkl')

In [5]:
# Training parameters
nb_epoch = 150
batch_size = 24
samples_per_epoch = 500
N_seq_val = 140  # number of sequences to use for validation

In [6]:
# Model parameters
n_channels, im_height, im_width = (7, 20, 40)
input_shape = (n_channels, im_height, im_width) if K.image_data_format() == 'channels_first' else (im_height, im_width, n_channels)
stack_sizes = (n_channels, 48, 96)
R_stack_sizes = stack_sizes
A_filt_sizes = (3, 3)
Ahat_filt_sizes = (3, 3, 3)
R_filt_sizes = (3, 3, 3)
layer_loss_weights = np.array([1., 0., 0.])  # weighting for each layer in final loss; "L_0" model:  [1, 0, 0, 0], "L_all": [1, 0.1, 0.1, 0.1]
layer_loss_weights = np.expand_dims(layer_loss_weights, 1)
nt = 24  # number of timesteps used for sequences in training
time_loss_weights = 1./ (nt - 1) * np.ones((nt,1))  # equally weight all timesteps except the first
time_loss_weights[0] = 0

In [7]:
prednet = PredNet(stack_sizes, R_stack_sizes,
                  A_filt_sizes, Ahat_filt_sizes, R_filt_sizes,
                  output_mode='error', return_sequences=True)

In [8]:
inputs = Input(shape=(nt,) + input_shape)
errors = prednet(inputs)  # errors will be (batch_size, nt, nb_layers)
errors_by_time = TimeDistributed(Dense(1, trainable=False), weights=[layer_loss_weights, np.zeros(1)], trainable=False)(errors)  # calculate weighted error by layer
errors_by_time = Flatten()(errors_by_time)  # will be (batch_size, nt)
final_errors = Dense(1, weights=[time_loss_weights, np.zeros(1)], trainable=False)(errors_by_time)  # weight errors by time
model = Model(inputs=inputs, outputs=final_errors)
model.compile(loss='mean_absolute_error', optimizer='adam')

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 24, 20, 40, 7)     0         
_________________________________________________________________
pred_net_1 (PredNet)         (None, 24, 3)             1621448   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 24, 1)             4         
_________________________________________________________________
flatten_1 (Flatten)          (None, 24)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 25        
Total params: 1,621,477
Trainable params: 1,621,448
Non-trainable params: 29
_________________________________________________________________


In [10]:
train_generator = SequenceGenerator(train_file, train_sources, nt, batch_size=batch_size, shuffle=False)
val_generator = SequenceGenerator(val_file, val_sources, nt, batch_size=batch_size, N_seq=N_seq_val)

In [11]:
lr_schedule = lambda epoch: 0.001 if epoch < 75 else 0.0001    # start with lr of 0.001 and then drop to 0.0001 after 75 epochs
callbacks = [LearningRateScheduler(lr_schedule)]
if save_model:
    if not os.path.exists(WEIGHTS_DIR): os.mkdir(WEIGHTS_DIR)
    callbacks.append(ModelCheckpoint(filepath=weights_file, monitor='val_loss', save_best_only=True))

In [12]:
history = model.fit_generator(train_generator, steps_per_epoch=(samples_per_epoch / batch_size), 
                              epochs=nb_epoch, callbacks=callbacks,
                              validation_data=val_generator, validation_steps=N_seq_val / batch_size,
                              verbose=2, workers=0)

Epoch 1/150
 - 21s - loss: 0.9718 - val_loss: 0.9142
Epoch 2/150
 - 15s - loss: 0.9089 - val_loss: 0.8895
Epoch 3/150
 - 15s - loss: 0.9057 - val_loss: 0.8826
Epoch 4/150
 - 15s - loss: 0.8905 - val_loss: 0.8765
Epoch 5/150
 - 15s - loss: 0.9106 - val_loss: 0.8840
Epoch 6/150
 - 15s - loss: 0.8812 - val_loss: 0.8749
Epoch 7/150
 - 15s - loss: 0.9099 - val_loss: 0.8897
Epoch 8/150
 - 15s - loss: 0.8829 - val_loss: 0.8711
Epoch 9/150
 - 15s - loss: 0.9039 - val_loss: 0.8843
Epoch 10/150
 - 15s - loss: 0.8814 - val_loss: 0.8696
Epoch 11/150
 - 15s - loss: 0.8897 - val_loss: 0.8881
Epoch 12/150
 - 15s - loss: 0.8803 - val_loss: 0.8696
Epoch 13/150
 - 15s - loss: 0.8819 - val_loss: 0.8836
Epoch 14/150
 - 15s - loss: 0.8786 - val_loss: 0.8672
Epoch 15/150
 - 15s - loss: 0.8732 - val_loss: 0.8671
Epoch 16/150
 - 15s - loss: 0.8805 - val_loss: 0.8698
Epoch 17/150
 - 15s - loss: 0.8643 - val_loss: 0.8600
Epoch 18/150
 - 15s - loss: 0.8845 - val_loss: 0.8574
Epoch 19/150
 - 16s - loss: 0.8581 - 

In [13]:
if save_model:
    json_string = model.to_json()
    with open(json_file, "w") as f:
        f.write(json_string)